# CS 181 - Project 2: IMDb Database

In [1]:
'''
CS 181 Project 2
Authors: Devan Plaga & Amna Khalid
Date: 10/25/2019
'''

import json
from sqlalchemy import create_engine
import pandas as pd
from IPython.display import display

In [2]:
def main():
    '''
    Sets up SQLAlchemy connection, executes queries, displays data, and exports data to several files.
    '''
    
    # Setup engine and connection
    database = 'imdb'
    creds = getCreds("creds.json", "mysql", defaults={'user': 'studen_j1', 
                                                  'password': 'studen_j1'})
    engine, connection, cstring = db_setup(creds['user'], creds['password'], database)
    
    # Setup queries and column names in a dictionary
    # Each value is a tuple in the form of (query, column_names) where
    # `query` is the query sent to the server while `column_names` is a tuple of column names to be written to a file
    queries = {
        'genres': ('''SELECT genre, AVG(averageRating) AS avg_rating FROM titles NATURAL JOIN genres NATURAL JOIN ratings
            GROUP BY genre ORDER BY avg_rating;''', ('genre', 'averageRating')),
        'ratings': ('''SELECT startYear, AVG(averageRating), COUNT(*) FROM titles NATURAL JOIN ratings
            GROUP BY startYear ORDER BY startYear;''', ('year', 'avg_rating', 'num_titles')),
        'directors': ('''SELECT s.num_directors, COUNT(*) FROM (SELECT COUNT(*) AS num_directors FROM titles
            NATURAL JOIN principles WHERE category = \'director\' GROUP BY titleID)s
            GROUP BY s.num_directors ORDER BY s.num_directors;''', ('num_directors', 'num_movies')),
        'durations': ('''SELECT primaryTitle, endYear - startYear AS duration FROM titles
            WHERE endYear IS NOT NULL ORDER BY duration DESC;''', ('title', 'duration')),
        'extreme_ratings': ('''SELECT primaryTitle, averageRating, startYear FROM titles NATURAL JOIN ratings
            WHERE startYear IN (1891, 1902) ORDER BY averageRating DESC;''', ('title', 'averageRating', 'year'))
    }
    
    # Create an empty dictionary to hold the received data
    data = {}
    
    # Iterate through and execute every query
    for entry in queries:
        # Get data and export to file
        data[entry] = resolve_query(*queries[entry], entry, connection)
        
        # Display data
        print(entry)
        display(data[entry].head())
    
    # Close the SQL connection
    connection.close()

In [3]:
def resolve_query(query, colnames, filename, connection):
    '''Executes SQL query, saves data in a dataframe, and exports data to a file
    Parameters:
        query: The SQL query to be executed
        colnames: The column names for the dataframe
        filename: The name of the file to write to (without extension)'''
    
    # Get data from query
    data = connection.execute(query).fetchall()
    # Store data in dataframe
    df = pd.DataFrame(data, columns=colnames)
    # Export data to csv file
    df.to_csv(filename + '.csv')
    return df

In [4]:
# This entire cell is borrowed from previous homework
def getCreds(filename, subset, defaults = {}):
    """ Use `filename` to look for a file containing a json-encoded dictionary
        of credentials.  If the file is successfully found and contains valid
        json, return the sub-dictionary based on `subset`.  If the file is not
        found, is not accessible, has improper encoding, or if the subset is 
        not present in the dictionary, return the given defaults.
    """
    try:
        with open(filename, 'r') as file:
            D = json.load(file)
            if D[subset]:
                return D[subset]
            else:
                return defaults
    except:
        return defaults

def db_setup(user, password, database):
    ''' Create a SQL engine and connection to the Denison SQL server and return them.
        Parameters:
            user: The user to connect to the database with
            password: The password to connect to the database with
            database: The database to connect to
        Returns: The engine, connection, and connection string used'''
    
    # Create connection string template and fill in values
    template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/{}'
    cstring = template.format(user, password, database)
    
    # Create and return engine and connection objects
    e = create_engine(cstring)
    c = e.connect()
    return e, c, cstring

In [5]:
main()

genres


,genre,averageRating
0,Horror,6.034808
1,Adult,6.046667
2,Reality-TV,6.286087
3,Thriller,6.378233
4,Sci-Fi,6.515527


ratings


,year,avg_rating,num_titles
0,1878.0,7.400000,1
1,1888.0,7.200000,2
2,1891.0,4.900000,1
3,1893.0,6.200000,1
4,1894.0,6.066667,3


directors


,num_directors,num_movies
0,1,34162
1,2,1941
2,3,318
3,4,63
4,5,36


durations


,title,duration
0,Guiding Light,57.0
1,As the World Turns,54.0
2,Steptoe and Son,54.0
3,One Life to Live,45.0
4,Rocky and His Friends,41.0


extreme_ratings


,title,averageRating,year
0,A Trip to the Moon,8.2,1902.0
1,Newark Athlete,4.9,1891.0


## SQL Query Explanation

Note: All joined tables can and have been naturally joined.

#### Query #1 - Average Rating of Each Genre:
`SELECT genre, AVG(averageRating) AS avg_rating FROM titles NATURAL JOIN genres NATURAL JOIN ratings
GROUP BY genre ORDER BY avg_rating;`

In this query we are trying to find the average rating of all entries in each genre. We can get the required data by joining the titles, genres, and ratings tables. Additionally, we want to average the ratings for each genre, so we group by genre and then sort by the average rating.

#### Query #2 - Average Ratings by Year:
`SELECT startYear, AVG(averageRating), COUNT(*) FROM titles NATURAL JOIN ratings
GROUP BY startYear ORDER BY startYear;`

In this query we are trying to find the average rating of all entries in every available year. We also record the total number of entries in each year as that may be useful. The required data can be acquired by joining the titles and ratings tables. Next, we need to group by the year so we can get the average rating of all entries in each year. Finally, we sort the data by year for convenience.

#### Query #3 - Number of Directors by Number of Productions:
`SELECT s.num_directors, COUNT(*) FROM (SELECT COUNT(*) AS num_directors FROM titles
NATURAL JOIN principles WHERE category = 'director' GROUP BY titleID)s
GROUP BY s.num_directors ORDER BY s.num_directors;`

In this query we are trying to find how many directors make each production. This query contains a subquery to initially count the number of directors in each production by joining the titles and principles tables. The subquery specifically only includes people from the 'director' category and groups by the production. From this subquery, we group and order by the number of directors and count the number of entries in each group, thereby getting the number of productions for each number of directors.

#### Query #4 - Duration of Each Production With a Duration:
`SELECT primaryTitle, endYear - startYear AS duration FROM titles
WHERE endYear IS NOT NULL ORDER BY duration DESC;`

In this query we are trying to find the duration in years of each production that has an end date. The duration is determined by subtracting the start year by the end year of each production. We only need to use the titles table and we filter it to only include values with an end date. Finally, we sort by descending duration.

#### Follow-up Query - The Ratings of All Productions in the Min and Max Average Rated Years:
`SELECT primaryTitle, averageRating, startYear FROM titles NATURAL JOIN ratings
WHERE startYear IN (1891, 1902) ORDER BY averageRating DESC;`

This query finds the title, rating, and year of every production in the two years that were discovered in query #2 to have the minimum and maximum average ratings. The data we need can be acquired by joining the titles and ratings tables. Then we filter it to only include productions in the minimum and maximum rated years (1891 and 1902, respectively) and order them by that average rating.

## Code Explanation

The program starts by creating a `sqlalchemy` connection string, engine, and connection. Next, it saves all of the queries and column names in a dictionary so everything can be iterated through later. Finally, it goes through the process of executing the queries and saving and displaying all of the data received.

## Data Exploration

### Q1. Which genre in our IMDb data has the most ratings?

We explore the relationship between the average ratings and different available genres for each of the entries in the IMDb Database by representing the statistics on a bar graph.

We find that the most rated genre according to our IMDb Database is the documentary genre followed by animation which is further followed by history. The high ratings of the documentary genre can be due to people’s greater interest in real life events more than fiction. Animation can be explained by the audience being mostly adolescents, teens and young adults which makes that genre more preferable than the others. History can also be explained with the interest in past events by new or older generations.

The lowest rated genre on our graph is horror. The horror genre targets a specific audience like teens or young adults which limits more to those who can actually sit through a horror movie. On the other hand, the highly rated genres target very broad audiences, which can explain the difference in ratings.
<img src="genre_rating.png">

### Q2. Which years have the minimum and maximum average rated IMDb entries?

We wanted to see how the average ratings are changing for the different entries over the years from 1878 to 2018. We explore the relationship using a line graph. We can see that the two extreme points for average ratings on our graph are for the years 1891, which had an average rating of 4.9, and 1902, which had an average rating of 8.2.

The minimum rated year in our data shows that the only recorded rating we have for it is for the *Newark Athlete*, which is an American short silent film directed and produced by William Kennedy Dickson. The low ratings for it can be explained by it not having an interesting enough genre or it being an art form that is too complicated for the general population to understand. 

The maximum rated year in our data shows that the only recorded rating is for *A Trip to the Moon*, which is a French adventure film directed by Georges Méliès. The film was famous and had very high ratings because of the idea of going to the moon. The actual act of going to the moon had not really taken place yet and this movie was wild representation of what could happen.

The reason these two years were extreme points in the data was partially due to not having any other data in those years. Statistically speaking, the less data that is averaged, the more likely it is for the average to be farther away from other averages of similar data due to outliers. Since the two films were the only entries in their respective years, the average ended up being simply their own ratings, and thus more extreme than any other year listed.
<img src="average_rating_by_year.png">

### Q3. How many directors does each entry in our IMDb data has?

We wanted to see how many directors each of the IMDb entries usually have. It was an interesting graph to look at because we can see that 34,162 of the entries had only one director, for a total that is greater than all the other director counts summed up. We can see that the second most common number of directors is 2 directors, which has 1,941 IMDb entries.

You cannot really see any other bars until you actually exclude the number of entries that have one or two directors only. After that we can see that the greatest number of directors an entry has is 6 for only 75 entries. (Go ahead and actually explore the data yourself by excluding data!) It is also interesting that there are more entries with 6 directors than with 4 or 5 directors, after there being so many entries with just 1 or 2 directors.

The fact that most entries have only one director is probably because people find it difficult to work with other people in a job that requires you to manage and be in control of how the final output is supported to look. Or maybe it is the fact that studios do not want to spend their budget on hiring more directors when they can spend that money on something that can help their production in some other way.
<img src="num_directors.png">
<img src="num_directors_zoomed.png">

### Q4. How long are the longest running programs on the IMDb Database and what are their names?

We explored the longest running time of each IMDb entry by subtracting the start year from the end year. We represented this information using a bar graph.

The longest running program was *Guiding Light*, which had been airing for a total of 57 years. The second longest running programs are *Steptoe and Son* and *As the World Turns*, which each have been airing for 54 years. This is followed by *One Life to Live*, which had been on air for 45 years.

An interesting fact is the entries that have the longest running are usually popular soap operas, kid TV shows, or sitcoms. There are also some wildly popular talk shows, for example *The Oprah Winfrey Show*, which was on for 27 years.

This data leads us to the conclusion that these particular types of TV are popular forms of entertainment which led to the productions running for years.
<img src="long_running.png">